Fuentes: https://medium.com/nlplanet/fine-tuning-distilbert-on-senator-tweets-a6f2425ca50e

#### **Instalar Modulos**

conda install datasets=="2.20.0"

conda install transformers=="4.40.1"

conda install numpy=="1.26.4" # La última versión no funciona bien


In [1]:
# Data processing
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import copy

import time
import datetime

from sklearn.metrics import confusion_matrix, cohen_kappa_score

from datasets import Dataset,  DatasetDict

import optuna
from optuna.artifacts import FileSystemArtifactStore, upload_artifact

# Modeling
import torch
from torch.utils.data import DataLoader
from transformers import DistilBertTokenizerFast, DataCollatorWithPadding, AutoModelForSequenceClassification, AdamW, get_scheduler

# Progress bar
from tqdm.auto import tqdm

from UA_MDM_LDI_II.tutoriales.utils import plot_confusion_matrix, get_artifact_filename

from joblib import load, dump

# Verificamos que CUDA está funcional
torch.cuda.is_available()

/home/aa/miniconda3/envs/ldi2_cuda_trf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

**Bajamos el modelo**

In [2]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/home/aa/miniconda3/envs/ldi2_cuda_trf/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


**Armado de los Datasets**

In [3]:
# Paths
BASE_DIR = './'
PATH_TO_TRAIN = os.path.join(BASE_DIR, "input/petfinder-adoption-prediction/train/train.csv")
#PATH_TO_IMAGES_DIR = os.path.join(BASE_DIR, "input/petfinder-adoption-prediction/train_images")
PATH_TO_TEMP_FILES = os.path.join(BASE_DIR, "UA_MDM_LDI_II/work/optuna_temp_artifacts")
PATH_TO_OPTUNA_ARTIFACTS = os.path.join(BASE_DIR, "UA_MDM_LDI_II/work/optuna_artifacts")

# Parametros y variables
SEED = 42
TEST_SIZE = 0.2

BATCH_SIZE = 64

MODEL_NAME = '06 Bert'

MODEL_VERSION = '1.0.1'

In [4]:
# Cargar los datos
df = pd.read_csv(PATH_TO_TRAIN)
df = df[df['Description'].notnull()]
df['labels'] = df["AdoptionSpeed"]

# Dividir los datos usando sklearn
train_df, test_df = train_test_split(df, test_size=TEST_SIZE, random_state=SEED, stratify=df.AdoptionSpeed)

#study_lgb = optuna.create_study(direction='maximize',
#                            storage="sqlite:///db.sqlite3",  # Specify the storage URL here.
#                            study_name="04 - LGB Multiclass CV",
#                           load_if_exists = True)

#lgb_test_dataset = load(os.path.join(PATH_TO_OPTUNA_ARTIFACTS,get_artifact_filename(study_lgb,'test')))

#train_df = df[~df.PetID.isin(lgb_test_dataset.PetID)].reset_index(drop=True)
#test_df = df[df.PetID.isin(lgb_test_dataset.PetID)].reset_index(drop=True)

# Convertir a Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Combinar en un DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'val': test_dataset
})

# Codificar la columna de etiquetas como clases
dataset = dataset.class_encode_column('labels')

# Hacer una lista de columnas para remover antes de la tokenización
cols_to_remove = [col for col in dataset["train"].column_names if col != 'labels']
print(cols_to_remove)

Casting to class labels: 100%|██████████| 2996/2996 [00:00<00:00, 313753.33 examples/s]

['Type', 'Name', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2', 'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed', 'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'RescuerID', 'VideoAmt', 'Description', 'PetID', 'PhotoAmt', 'AdoptionSpeed', '__index_level_0__']


In [5]:
# Tokenize and encode the dataset
def tokenize(batch):
    from transformers import DistilBertTokenizerFast
    tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
    tokenized_batch = tokenizer(batch["Description"], padding=True, truncation=True, max_length=512)
    return tokenized_batch

dataset_enc = dataset.map(tokenize, batched=True, remove_columns=cols_to_remove, num_proc=4)

# Set dataset format for PyTorch
dataset_enc.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

# Check the output
print(dataset_enc["train"].column_names)
     


Map (num_proc=4):   0%|          | 0/11984 [00:00<?, ? examples/s]/home/aa/miniconda3/envs/ldi2_cuda_trf/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/aa/miniconda3/envs/ldi2_cuda_trf/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/aa/miniconda3/envs/ldi2_cuda_trf/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/

['labels', 'input_ids', 'attention_mask']


In [6]:
# Instantiate a data collator with dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Create data loaders for to reshape data for PyTorch model
train_dataloader = DataLoader(
    dataset_enc["train"], shuffle=True, batch_size=BATCH_SIZE, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    dataset_enc["val"], batch_size=BATCH_SIZE, collate_fn=data_collator
)

In [7]:
test_sample_ids =[i for i in test_df.PetID] 

In [8]:
# Dynamically set number of class labels based on dataset
num_labels = dataset["train"].features['labels'].num_classes
print(f"Number of labels: {num_labels}")

# Load model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", 
                                                           num_labels=num_labels)

Number of labels: 5


/home/aa/miniconda3/envs/ldi2_cuda_trf/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:

# Set the device automatically (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Move model to device
model.to(device)

cuda


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [10]:
def train_val(model, dataloaders, datasets, device, num_epochs=4, lr=0.001, trial=None):
    
    since = time.time()

    # Create the optimizer
    optimizer = AdamW(model.parameters(), lr=lr)

    # Further define learning rate scheduler
    num_training_batches = len(train_dataloader)
    num_training_steps = num_epochs * num_training_batches
    lr_scheduler = get_scheduler(
        "linear",                   # linear decay
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )


    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_kappa =  -999

    train_losses = []
    val_losses = []

    try:
        previous_best = study.best_value
    except:
        previous_best = -999


    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        kappa_labels_true = []
        kappa_labels_predicted = []
        output_scores = []

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for batch in tqdm(dataloaders[phase]):
                batch = batch.to(device)
                #inputs = inputs.to(device)
                labels = batch.labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                # Track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(**batch)
                    loss = outputs.loss

                    preds = torch.nn.functional.softmax(outputs.logits, dim=-1)
                    preds_labels = torch.argmax(preds, dim=-1)


                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    elif phase == 'val':
                        kappa_labels_true.extend(labels.cpu().numpy().tolist())
                        kappa_labels_predicted.extend(preds_labels.cpu().numpy().tolist())
                        outputs_np = preds.cpu().numpy()
                        output_scores.extend([outputs_np[i,:] for i in range(outputs_np.shape[0])])

                # Statistics
                running_loss += loss.item() * labels.size(0)
                running_corrects += torch.sum(preds_labels == labels.data)
                
                #END OF BATCH

            epoch_loss = running_loss / len(datasets[phase])
            epoch_acc = running_corrects.double() / len(datasets[phase])
            
            if phase == 'train':
                train_losses.append(epoch_loss)
                kappa_score = np.nan
            else:
                val_losses.append(epoch_loss)
                kappa_score = cohen_kappa_score(kappa_labels_true,
                                  kappa_labels_predicted,
                                  weights = 'quadratic')
                    


            print(f'{phase.title()} Loss: {epoch_loss:.4f} Acc: {epoch_acc*100:.2f}% Kappa: {kappa_score:.3f}')

            # If this is the best Epoch so far -> Deep copy the model
            if phase == 'val' and kappa_score > best_kappa:
                best_acc = epoch_acc
                best_kappa = kappa_score
                best_model_wts = copy.deepcopy(model.state_dict())


                #Best Epoch within a trial and better than previous trials
                if trial is not None and best_kappa > previous_best:

                    #Save test dataset with predictions
                    predicted_filename = os.path.join(PATH_TO_TEMP_FILES,f'test_{trial.study.study_name}_{trial.number}.joblib')
                    predicted_df = pd.DataFrame({'PetID':test_sample_ids,
                                'pred':output_scores}).merge(test_df, on='PetID')
                    dump(predicted_df, predicted_filename)

                    #Generate and save CM 
                    cm_filename = os.path.join(PATH_TO_TEMP_FILES,f'cm_{trial.study.study_name}_{trial.number}.jpg')
                    plot_confusion_matrix(kappa_labels_true,kappa_labels_predicted).write_image(cm_filename)

            #END OF PHASE

        #END OF EPOCH

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.2f}%'.format(best_acc * 100))

    # Load best model weights
    model.load_state_dict(best_model_wts)

    # Save in optuna trial the best test dataset, cm and model weights
    if trial is not None and best_kappa > previous_best:
        upload_artifact(trial, predicted_filename, artifact_store)   

        upload_artifact(trial, cm_filename, artifact_store)

        file_name = f'{MODEL_NAME}_{MODEL_VERSION}_{trial.number}.pth'
        model_path = os.path.join(PATH_TO_TEMP_FILES, file_name)
        torch.save(model, model_path) # Podemos guardar solo los pesos si queremos: best_model.state_dict()
        upload_artifact(trial, model_path, artifact_store)

    return model,best_kappa



In [11]:

# Dynamically set number of class labels based on dataset
num_labels = dataset["train"].features['labels'].num_classes
print(f"Number of labels: {num_labels}")


Number of labels: 5


In [ ]:

best_model,_ = train_val(model,
                       dataloaders={'train': train_dataloader, 
                                    'val': eval_dataloader}, 
                       datasets=dataset_enc, 
                       device=device, 
                       lr = 5e-5,
                       num_epochs=15)


In [ ]:
# Guardo el modelo
run_id = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
file_name = f'{MODEL_NAME}_{MODEL_VERSION}_{run_id}.pth'
model_path = os.path.join(PATH_TO_TEMP_FILES, file_name)
torch.save(best_model, model_path) # Podemos guardar solo los pesos si queremos: best_model.state_dict()
print(f'Modelo guardado en {model_path}')

Modelo guardado en ./UA_MDM_LDI_II/work/optuna_temp_artifacts/06 Bert_1.0.0_20240702_135807.pth


In [12]:
artifact_store = FileSystemArtifactStore(base_path=PATH_TO_OPTUNA_ARTIFACTS)


def optuna_train(trial):

    epochs = trial.suggest_int('epochs', 1, 2)

    lr = trial.suggest_float('lr', 0.00001, 0.1, log=True)

    _,best_score = train_val(model, 
                       dataloaders={'train': train_dataloader, 
                                    'val': eval_dataloader}, 
                       datasets=dataset_enc, 
                       device=device, 
                       num_epochs=epochs,
                       lr=lr,
                       trial=trial)


    return(best_score)

/tmp/ipykernel_84980/3100232574.py:1: ExperimentalWarning: FileSystemArtifactStore is experimental (supported from v3.3.0). The interface can change in the future.
  artifact_store = FileSystemArtifactStore(base_path=PATH_TO_OPTUNA_ARTIFACTS)


In [13]:
study = optuna.create_study(direction='maximize',
                            storage="sqlite:///db.sqlite3",  # Specify the storage URL here.
                            study_name=f'{MODEL_NAME}_{MODEL_VERSION}',
                            load_if_exists = True)
study.optimize(optuna_train, n_trials=5)

[I 2024-07-02 17:24:55,603] Using an existing study with name '06 Bert_1.0.1' instead of creating a new one.
/home/aa/miniconda3/envs/ldi2_cuda_trf/lib/python3.9/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 0/0
----------


  3%|▎         | 5/188 [00:03<02:02,  1.50it/s]